** This notebook has a few of my favorite interactive examples. **

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='c1bdae733682f8a390f848bc6cb0dbbf9ea98602', drake_version='0.25.0', drake_build='releases')

from manipulation import running_as_notebook

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import Textarea
from functools import partial
import meshcat

import pydrake.all
from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
from manipulation.scenarios import AddIiwa, AddWsg, AddShape, AddTwoLinkIiwa, AddRgbdSensors
from manipulation.jupyter_widgets import MakePoseSlidersThatPublishOnCallback, MakeJointSlidersThatPublishOnCallback

pydrake.common.set_log_level("warn");

# Visualizing Jacobian-based control

In [ ]:
import numpy as np
from IPython.display import display
from ipywidgets import FloatSlider, Textarea, Layout

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph, RigidTransform, ConnectMeshcatVisualizer,
                         JacobianWrtVariable, Parser, FixedOffsetFrame,
                         PiecewisePolynomial, MathematicalProgram, LinearConstraint, Solve)
from manipulation.utils import FindResource
from manipulation.meshcat_utils import plot_mathematical_program


# This one is specific to this notebook, but I'm putting it in the header to make it less distracting.
def Visualizer(MakeMathematicalProgram):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1.0)
    twolink = AddTwoLinkIiwa(plant, q0=[0.0, 0.0])
    hand = plant.GetFrameByName("iiwa_link_ee")
    plant.Finalize()

    visualizer = ConnectMeshcatVisualizer(builder,
                                 scene_graph,
                                 zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    visualizer.vis.delete()
    #visualizer.vis["/Background"].set_property('visible',False)
    visualizer.vis["/Background"].set_property('top_color', [0, 0, 0])
    visualizer.vis["/Background"].set_property('bottom_color', [0, 0, 0])
    visualizer.vis["/Grid"].set_property('visible',False)
    visualizer.load()

    jacobian = Textarea(value="", description="J_G: ", layout={'width':'200pm','height':'50px'}, style={'description_width':'initial'})
    display(jacobian)

    X, Y = np.meshgrid(np.linspace(-5, 5, 35), np.linspace(-5, 5, 31))

    def visualize(q, v_Gdesired=[1.0, 0.0], t=None):
        if t:
            context.SetTime(t)
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        J_G = plant.CalcJacobianTranslationalVelocity(plant_context, JacobianWrtVariable.kQDot, hand, [0,0,0], plant.world_frame(), plant.world_frame())
        J_G = J_G[[0,2],:]  # Ignore Y.
        jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.2f}".format(x)})

        prog = MakeMathematicalProgram(q, J_G, v_Gdesired)
        result = Solve(prog)
        v = visualizer.vis["QP"]
        plot_mathematical_program(v, prog, X, Y, result=result)
        # TODO: Add set_object to meshcat.Animation
        if False: # meshcat._is_recording:
            with visualizer._animation.at_frame(
                    v, visualizer._recording_frame_num) as m:
                plot_mathematical_program(m, prog, X, Y, result=result)

    return visualize, visualizer

def MakeMathematicalProgram(q, J_G, v_Gdesired):
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(2, 'v')
    v_max = 3.0 

    error = J_G.dot(v) - v_Gdesired
    prog.AddCost(error.dot(error))
    prog.AddBoundingBoxConstraint(-v_max, v_max, v)

    return prog

visualize, _ = Visualizer(MakeMathematicalProgram)

q = [-np.pi/2.0+0.5, 1.0]
v_Gdesired = [0.5, 0.]
visualize(q, v_Gdesired)

In [ ]:
visualize, _ = Visualizer(MakeMathematicalProgram)

v_Gdesired = [1.0, 0.0]
T = 2.
qtraj = PiecewisePolynomial.FirstOrderHold(
    [0, T], np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [2., -2.]]))
visualize(qtraj.value(0), v_Gdesired)

def _t_callback(change):
    visualize(qtraj.value(change.new), v_Gdesired)

slider = FloatSlider(value=0, min=0, max=T, step=0.05, continuous_update=True, description="t", layout=Layout(width="'100'"))
slider.observe(_t_callback, names='value')
display(slider)

# Scoring grasp candidates

In [ ]:
import open3d as o3d
from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator, ProcessModelDirectives, LoadModelDirectives
)
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.utils import AddPackagePaths

def grasp_candidate_cost(plant_context, cloud, plant, scene_graph, scene_graph_context, adjust_X_G=False, textbox=None, meshcat=None):
    body = plant.GetBodyByName("body")
    X_G = plant.GetFreeBodyPose(plant_context, body)

    # Transform cloud into gripper frame
    X_GW = X_G.inverse()
    pts = np.asarray(cloud.points).T
    p_GC = X_GW.multiply(pts)

    # Crop to a region inside of the finger box.
    crop_min = [-.05, 0.1, -0.00625]
    crop_max = [.05, 0.1125, 0.00625]
    indices = np.all((crop_min[0] <= p_GC[0,:], p_GC[0,:] <= crop_max[0],
                      crop_min[1] <= p_GC[1,:], p_GC[1,:] <= crop_max[1],
                      crop_min[2] <= p_GC[2,:], p_GC[2,:] <= crop_max[2]), 
                     axis=0)

    if meshcat:
        draw_points(meshcat["points"], pts[:, indices], [1., 0, 0], size=0.01)

    if adjust_X_G and np.sum(indices)>0:
        p_GC_x = p_GC[0, indices]
        p_Gcenter_x = (p_GC_x.min() + p_GC_x.max())/2.0
        X_G.set_translation(X_G.translation() + X_G.rotation().multiply([p_Gcenter_x, 0, 0]))
        plant.SetFreeBodyPose(plant_context, body, X_G)
        X_GW = X_G.inverse()

    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    # Check collisions between the gripper and the sink
    if query_object.HasCollisions():
        cost = np.inf
        if textbox:
            textbox.value = "Gripper is colliding with the sink!\n"
            textbox.value += f"cost: {cost}"
        return cost

    # Check collisions between the gripper and the point cloud
    margin = 0.0  # must be smaller than the margin used in the point cloud preprocessing.
    for pt in cloud.points:
        distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=margin)
        if distances:
            cost = np.inf
            if textbox:
                textbox.value = "Gripper is colliding with the point cloud!\n"
                textbox.value += f"cost: {cost}"
            return cost


    n_GC = X_GW.rotation().multiply(np.asarray(cloud.normals)[indices,:].T)

    # Penalize deviation of the gripper from vertical.
    # weight * -dot([0, 0, -1], R_G * [0, 1, 0]) = weight * R_G[2,1]
    cost = 20.0*X_G.rotation().matrix()[2, 1]

    # Reward sum |dot product of normals with gripper x|^2
    cost -= np.sum(n_GC[0,:]**2)

    if textbox:
        textbox.value = f"cost: {cost}\n"
        textbox.value += "normal terms:" + str(n_GC[0,:]**2)
    return cost


def process_point_cloud(diagram, context, cameras, bin_name):
    """A "no frills" version of the example above, that returns the down-sampled point cloud"""
    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    # Compute crop box.
    bin_instance = plant.GetModelInstanceByName(bin_name)
    bin_body = plant.GetBodyByName("bin_base", bin_instance)
    X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
    margin = 0.001  # only because simulation is perfect!
    a = X_B.multiply([-.22+0.025+margin, -.29+0.025+margin, 0.015+margin])
    b = X_B.multiply([.22-0.1-margin, .29-0.025-margin, 2.0])
    crop_min = np.minimum(a,b)
    crop_max = np.maximum(a,b)

    # Evaluate the camera output ports to get the images.
    merged_pcd = o3d.geometry.PointCloud()
    for c in cameras:
        point_cloud = diagram.GetOutputPort(f"{c}_point_cloud").Eval(context)
        pcd = create_open3d_point_cloud(point_cloud)

        # Crop to region of interest.
        pcd = pcd.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=crop_min,
                                                max_bound=crop_max))    

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(c)
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd.orient_normals_towards_camera_location(X_C.translation())
        
        # Merge point clouds.
        merged_pcd += pcd

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    return merged_pcd.voxel_down_sample(voxel_size=0.005)

def make_environment_model(directive=None, draw=False, rng=None, num_ycb_objects=0, bin_name="bin0"):
    # Make one model of the environment, but the robot only gets to see the sensor outputs.
    if not directive:
        directive = FindResource("models/two_bins_w_cameras.yaml")

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
    parser = Parser(plant)
    AddPackagePaths(parser)
    ProcessModelDirectives(LoadModelDirectives(directive), plant, parser)

    for i in range(num_ycb_objects):
        object_num = rng.integers(len(ycb))
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()
    AddRgbdSensors(builder, plant, scene_graph)

    if draw:
        meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment")

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    if num_ycb_objects > 0:
        generator = RandomGenerator(rng.integers(1000))  # this is for c++
        plant_context = plant.GetMyContextFromRoot(context)
        bin_instance = plant.GetModelInstanceByName(bin_name)
        bin_body = plant.GetBodyByName("bin_base", bin_instance)
        X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
        z = 0.1
        for body_index in plant.GetFloatingBaseBodies():
            tf = RigidTransform(
                    UniformlyRandomRotationMatrix(generator),  
                    [rng.uniform(-.15,.15), rng.uniform(-.2, .2), z])
            plant.SetFreeBodyPose(plant_context, 
                                plant.get_body(body_index),
                                X_B.multiply(tf))
            z += 0.1

        simulator = Simulator(diagram, context)
        simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
    elif draw:
        meshcat.load()
        diagram.Publish(context)


    return diagram, context

def grasp_score_inspector():
    v = meshcat.Visualizer(zmq_url=zmq_url)
    v.delete()
    environment, environment_context = make_environment_model(directive = FindResource("models/clutter_mustard.yaml"))

    # Another diagram for the objects the robot "knows about": gripper, cameras, bins.  Think of this as the model in the robot's head.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    AddPackagePaths(parser)
    ProcessModelDirectives(LoadModelDirectives(FindResource("models/clutter_planning.yaml")), plant, parser)
    plant.Finalize()
    
    v = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="planning")
    v.load()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    cloud = process_point_cloud(environment, environment_context, ["camera0", "camera1", "camera2"], "bin0")
    draw_open3d_point_cloud(v.vis["cloud"], cloud, size=0.003)

    textbox = Textarea(description="cost info", layout={'width': '800px', 'height': '100px'})
    display(textbox)

    lower_limit = [-np.pi, -np.pi/4., -np.pi/4., -1, -1, 0]
    upper_limit = [0, np.pi/4., np.pi/4., 1, 1, 1]
    sliders = MakeJointSlidersThatPublishOnCallback(
        plant, diagram, context, my_callback=partial(
            grasp_candidate_cost, cloud=cloud, plant=plant, scene_graph=scene_graph,
            scene_graph_context=scene_graph.GetMyContextFromRoot(context),
            textbox=textbox, meshcat=v.vis["cost"]),
        lower_limit=lower_limit, upper_limit=upper_limit)
    sliders[0].value = -np.pi/2.0
    sliders[3].value = -0.05
    sliders[4].value = -.5
    sliders[5].value = 0.25

grasp_score_inspector()

# Inverse kinematics as an optimization

In [ ]:
def teleop_inverse_kinematics(grasp_cylinder=True):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa, welded=True)
    cylinder = AddShape(plant, pydrake.geometry.Cylinder(0.02, 1.0), "cylinder")
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    cylinder_body = plant.GetBodyByName("cylinder", cylinder)
    cylinder_frame = plant.GetFrameByName("cylinder", cylinder)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(cylinder_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(cylinder_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        if grasp_cylinder:
            ik.AddPositionConstraint(gripper_frame, [0, 0.1, -0.02], cylinder_frame, [0, 0, -0.5], [0, 0, 0.5])
            ik.AddPositionConstraint(gripper_frame, [0, 0.1, 0.02], cylinder_frame, [0, 0, -0.5], [0, 0, 0.5])
        else:
            ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WC = RigidTransform(RollPitchYaw(np.pi/2.0, 0, 0), [0.5, 0, 0.5])
    s = MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WC)
    display(console)

# Set grasp_cylinder=False if you just want to antagonize the robot with a stick.
teleop_inverse_kinematics(grasp_cylinder=True)
